In [ ]:
!pip install -U efficientnet

# Import libraries and set seeds

In [ ]:
import os, sys, json
os.environ['PYTHONHASHSEED']='0'
import tensorflow as tf
tf.random.set_seed(42)
import efficientnet.tfkeras as efn
import cv2
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
np.random.seed(42)
import tensorflow.keras as keras
import albumentations
import pickle
# from google.colab.patches import cv2_imshow
from functools import partial
from kaggle_datasets import KaggleDatasets

# EDA

In [ ]:
labels = json.loads(open('/kaggle/input/cassava-leaf-disease-classification/label_num_to_disease_map.json', 'r').read())
print(labels)

In [ ]:
train = pd.read_csv('/kaggle/input/cassava-leaf-disease-classification/train.csv')
print(train.head())

## Check the imbalance of no. of samples in each class

In [ ]:
train.label.value_counts()

## Estimate class weight scales to compensate for class imbalance
Instead of resampling, I am going to scale the loss of each class by a respective class weight

In [ ]:
weights = np.array([1087, 2189, 2386, 13158, 2577])
print(np.sum(weights))

## How?
A quick way: Sklearn (https://scikit-learn.org/stable/modules/generated/sklearn.utils.class_weight.compute_class_weight.html)
Class weight = `total_samples/(samples_i * num_classes)`.

For 5 balanced classes, lets say total weight for all samples = 1.

Let each class have x samples, and there are total 5x samples. (In our case, total=21397).

$$  5x\  ->\ 1$$

$$    x\  ->\ ? $$
    
$$  x\  -> 1/5 (x\ samples\ scaled\ by\ 1/5)$$

Here, 0th class has 1087 samples
$$ X = \sum\limits_{i=1}^{5} {x} = 21397$$

$$  x0\  =\ (1807*X)/(21397)$$

$$  x\  ->\ 1/5$$

$$ x0\  ->\ ? $$

$$ x0\  ->\ 1807/(21397*5)$$

(I have tried my best to explain it in a few lines, reading a proper aticle/paper is recommended)

In [ ]:
cl_w = []
for i in weights:
    cl_w.append(np.sum(weights)/(5*i))

In [ ]:
cl_w

In [ ]:
norm_cw = np.sqrt(np.array(cl_w))

In [ ]:
class_weights = {0:norm_cw[0], 1:norm_cw[1], 2:norm_cw[2], 3:norm_cw[3], 4:norm_cw[4]}

In [ ]:
class_weights

## Visualization
Plot 5 samples of each class

In [ ]:
fig = plt.figure(figsize=(20,20))
count = 0
img_classes = {0:5, 1:5, 2:5, 3:5, 4:5}
for idx, i in enumerate(list(os.listdir('/kaggle/input/cassava-leaf-disease-classification/train_images/'))):
    img_class = train.loc[train['image_id']==i]['label'].tolist()[0]
    if img_classes[img_class] > 0:
        img_classes[img_class] -= 1
    else:
        continue
    ax = fig.add_subplot(5, 5, count+1)
    ax.set_title("Class = "+str(train.loc[train['image_id']==i]['label'].tolist()[0]))
    img = cv2.imread('/kaggle/input/cassava-leaf-disease-classification/train_images/'+i)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    plt.imshow(img)
    count += 1
    if count == 25:
        break

# Prepare dataset
A very simple tutorial from keras on how to use tfrecords: https://keras.io/examples/keras_recipes/tfrecord/

In [ ]:
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print("Device:", tpu.master())
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
except:
    strategy = tf.distribute.get_strategy()
print("Number of replicas:", strategy.num_replicas_in_sync)

In [ ]:
AUTOTUNE = tf.data.experimental.AUTOTUNE
GCS_PATH = KaggleDatasets().get_gcs_path()
print(GCS_PATH)
BATCH_SIZE = 128
IMAGE_SIZE = [512, 680]

In [ ]:
tfrecords = tf.io.gfile.glob(GCS_PATH + '/train_tfrecords/ld_train*.tfrec')

In [ ]:
FILENAMES = tfrecords
split_ind = int(0.9 * len(FILENAMES))
TRAINING_FILENAMES, VALID_FILENAMES = FILENAMES[:split_ind], FILENAMES[split_ind:]

In [ ]:
TRAINING_FILENAMES, VALID_FILENAMES

In [ ]:
def decode_image(image):
    """
        decode/read image and and cast to fp32
    """
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.cast(image, tf.float32)
    return image

def train_preprocess(image, label):
    """
        Data augmentations for training
    """
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_brightness(image, max_delta=0.2)
#     image = tf.image.random_contrast(image, lower=0.1, upper=0.2)
#     image = tf.image.random_saturation(image, 2, 4)
    image = tf.image.random_jpeg_quality(image, 90, 100)
    image = tf.image.per_image_standardization(image)
    image = tf.image.random_crop(image, [500, 500, 3])
    image = tf.image.resize(image, IMAGE_SIZE, method=tf.image.ResizeMethod.BILINEAR, preserve_aspect_ratio=False)
    return image, label

def val_preprocess(image, label):
    """
        Only image standardization for validation data
    """
    image = tf.image.per_image_standardization(image)
    image = tf.image.resize(image, IMAGE_SIZE, method=tf.image.ResizeMethod.BILINEAR, preserve_aspect_ratio=False)
    
    return image, label

In [ ]:
def read_tfrecord(example, labeled):
    tfrecord_format = (
        {
            "image": tf.io.FixedLenFeature([], tf.string),
            "target": tf.io.FixedLenFeature([], tf.int64),
        }
        if labeled
        else {"image": tf.io.FixedLenFeature([], tf.string),}
    )
    example = tf.io.parse_single_example(example, tfrecord_format)
    image = decode_image(example["image"])
    if labeled:
        label = tf.cast(example["target"], tf.int32)
        # One hot encode label
        label = tf.one_hot(indices=label, depth=5, on_value=1, off_value=0)
        return image, label
    return image

In [ ]:
def load_dataset(filenames, labeled=True, transforms=None):
    ignore_order = tf.data.Options()
    ignore_order.experimental_deterministic = False  # disable order, increase speed
    dataset = tf.data.TFRecordDataset(filenames)  # automatically interleaves reads from multiple files
    dataset = dataset.with_options(ignore_order)  # uses data as soon as it streams in, rather than in its original order
    dataset = dataset.map(partial(read_tfrecord, labeled=labeled), num_parallel_calls=AUTOTUNE)
    # returns a dataset of (image, label) pairs if labeled=True or just images if labeled=False
    return dataset

def get_dataset(filenames, labeled=True, transforms=None):
    dataset = load_dataset(filenames, labeled=labeled, transforms=transforms)
    dataset = dataset.shuffle(2048)
    dataset = dataset.prefetch(buffer_size=AUTOTUNE)
    dataset = dataset.map(transforms, num_parallel_calls=AUTOTUNE)
    dataset = dataset.batch(BATCH_SIZE)
    return dataset

In [ ]:
train_dataset = get_dataset(TRAINING_FILENAMES, labeled=True, transforms=train_preprocess)
valid_dataset = get_dataset(VALID_FILENAMES, labeled=True, transforms=val_preprocess)
full_dataset = get_dataset(FILENAMES, labeled=True, transforms=val_preprocess)

In [ ]:
train_dataset

In [ ]:
# lr scheduler
initial_learning_rate = 0.00025
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate, decay_steps=4, decay_rate=0.96, staircase=True
)

In [ ]:
dir(tf.keras.applications)

In [ ]:
def make_model():
    base_model = efn.EfficientNetB5(
        input_shape=(*IMAGE_SIZE, 3), include_top=False, weights='noisy-student'
    )

    base_model.trainable = True

    inputs = tf.keras.layers.Input([*IMAGE_SIZE, 3])
    # x = tf.keras.applications.densenet.preprocess_input(inputs)
    x = base_model(inputs)
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Dense(128, activation="relu")(x)
    x = tf.keras.layers.Dropout(0.2)(x)
    outputs = tf.keras.layers.Dense(5, activation="softmax")(x)

    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    
    # Use lr_scheduler as input to the optimizer
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule),
        loss=keras.losses.CategoricalCrossentropy(),
        metrics=['accuracy']
    )

    return model

## Define callbacks
For early stopping and saving checkpoints. I am not using LR on plateau as I have used a scheduler instead (I am not training for a lot of epochs and the network is pretrained)

In [ ]:
cp_path = "/kaggle/working/cp2_efn7_latest.h5"
callbacks = [keras.callbacks.ModelCheckpoint(cp_path, verbose=1, save_best_only=True, monitor='val_loss'),
             keras.callbacks.EarlyStopping(patience=3, verbose=True)]

In [ ]:
with strategy.scope():
    model = make_model()

In [ ]:
history = model.fit(
    train_dataset,
    epochs=15,
    validation_data=valid_dataset,
    callbacks=callbacks,
    class_weight=class_weights,
    verbose=1
)

One last epoch with all the data

In [ ]:
with strategy.scope():
    model = keras.models.load_model('/kaggle/working/cp2_efn7_latest.h5')
    

In [ ]:
model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
        loss=keras.losses.CategoricalCrossentropy(),
        metrics=['accuracy']
    )

In [ ]:
history = model.fit(
    train_dataset,
    epochs=1,
    validation_data=valid_dataset,
    callbacks=callbacks,
    class_weight=class_weights,
    verbose=1
)

In [ ]:
model.save('final_eb5.h5')

# Since final submission does not allow TPU's, the inference script will be separate